Cálculo de ETo (FAO-56 Penman-Monteith) con datos de NASA POWER

Calcular la Evapotranspiración de referencia (ETo) diaria usando la ecuación FAO-56 Penman-Monteith.
Procesar automáticamente archivos CSV por región (Periodo de Cultivo).
Generar nuevos CSV con las columnas originales más diagnósticos y la columna ET0.

Entradas:
Carpeta: /Datos/2010-2024 Periodo de Cultivo/
Archivos CSV por región y año/ciclo agrícola.

Salidas:
Carpeta: /Salidas/Periodo de Cultivo ETo/<Región>/
Archivos *-ETo.csv con columnas originales + columnas calculadas:
Tmean, es, ea, delta, P, gamma, Rns, Rnl, Rn, ET0, decada
Y columnas vacías: Kc, ETc, ETverde, ETazul (para compatibilidad con futuros cálculos de Huella Hídrica).

Notas:

Si alguna columna tiene nombre distinto (Día, Tmax, HR, Ux, Rs…), el código las renombra automáticamente.
PRECTOTCORR (precipitación) no es obligatoria para calcular ETo. Si no está, se agrega con valor 0.

In [2]:
from pathlib import Path
import pandas as pd
import numpy as np
import math
from datetime import datetime, timedelta

In [3]:
# --- Rutas en Yuca ---
RUTA_BASE    = Path("/lustre/home/mvalenzuela/Ocotillo/DataAqua")
RUTA_ENTRADA = RUTA_BASE / "Datos" / "2010-2024 Periodo de Cultivo"
RUTA_SALIDA  = RUTA_BASE / "Salidas_ETo12_con_uac_y_hh" / "Periodo de Cultivo ETo"
RUTA_SALIDA.mkdir(parents=True, exist_ok=True)

In [4]:
# --- Regiones con información de cultivo, fechas de cultivo, duracion de las etapas de crecimiento, ubicación y Kc (FAO) ---
REGIONES = {
    "Cajeme": {
        "lat": 27.507, "lon": -109.892, "z_m": 107,
        "cultivo": "Trigo",
        "calendario": {"start_mmdd": (10, 30), "end_mmdd": (4, 19)},
        "dur": (31, 47, 63, 31),
        "kc": {"ini": 0.30, "mid": 1.15, "end": 0.25}
    },
    "Etchojoa": {
        "lat": 26.91, "lon": -109.62, "z_m": 136,
        "cultivo": "Trigo",
        "calendario": {"start_mmdd": (10, 30), "end_mmdd": (4, 19)},
        "dur": (31, 47, 63, 31),
        "kc": {"ini": 0.30, "mid": 1.15, "end": 0.25}
    },
    "Ensenada": {
        "lat": 31.74, "lon": -116.55, "z_m": 583,
        "cultivo": "Fresa",
        "calendario": {"start_mmdd": (10, 1), "end_mmdd": (2, 20)},
        "dur": (29, 36, 57, 21),
        "kc": {"ini": 0.40, "mid": 0.90, "end": 0.75}
    },
    "Zapopan": {
        "lat": 20.72, "lon": -103.49, "z_m": 1572,
        "cultivo": "Maíz grano",
        "calendario": {"start_mmdd": (5, 15), "end_mmdd": (10, 22)},
        "dur": (29, 44, 59, 29),
        "kc": {"ini": 0.30, "mid": 1.20, "end": 0.35}
    },
    "Toluca": {
        "lat": 19.31, "lon": -99.61, "z_m": 2627,
        "cultivo": "Maíz grano blanco",
        "calendario": {"start_mmdd": (4, 20), "end_mmdd": (10, 6)},
        "dur": (31, 46, 62, 31),
        "kc": {"ini": 0.30, "mid": 1.20, "end": 0.35}
    },
    "Metepec": {
        "lat": 19.27, "lon": -99.63, "z_m": 2627,
        "cultivo": "Maíz grano blanco",
        "calendario": {"start_mmdd": (4, 20), "end_mmdd": (10, 6)},
        "dur": (31, 46, 62, 31),
        "kc": {"ini": 0.30, "mid": 1.20, "end": 0.35}
    },
    "Villa de Allende": {
        "lat": 19.38, "lon": -100.15, "z_m": 2460,
        "cultivo": "Avena forrajera en verde",
        "calendario": {"start_mmdd": (6, 15), "end_mmdd": (9, 12)},
        "dur": (16, 25, 33, 16),
        "kc": {"ini": 0.30, "mid": 1.15, "end": 0.80}  # final alto por corte (sin senescencia)
    },
}

In [5]:
# --- Rendimientos anuales (Para calcular HH) ---
RENDIMIENTOS_TON = {
    2010: {"Ensenada": 56.96, "Cajeme": 6.41, "Etchojoa": 6.41, "Zapopan": 7.25, "Villa de Allende": 15.96, "Toluca": 3.68, "Metepec": 4.44},
    2011: {"Ensenada": 46.70, "Cajeme": 6.30, "Etchojoa": 6.31, "Zapopan": 5.31, "Villa de Allende": 27.98, "Toluca": 3.07, "Metepec": 2.50},
    2012: {"Ensenada": 52.20, "Cajeme": 7.15, "Etchojoa": 7.13, "Zapopan": 7.49, "Villa de Allende": 25.81, "Toluca": 3.69, "Metepec": 3.50},
    2013: {"Ensenada": 60.34, "Cajeme": 7.19, "Etchojoa": 6.72, "Zapopan": 8.41, "Villa de Allende": 18.97, "Toluca": 4.33, "Metepec": 3.96},
    2014: {"Ensenada": 64.12, "Cajeme": 6.22, "Etchojoa": 5.99, "Zapopan": 8.43, "Villa de Allende": 19.82, "Toluca": 3.83, "Metepec": 4.10},
    2015: {"Ensenada": 32.64, "Cajeme": 5.11, "Etchojoa": 4.79, "Zapopan": 8.49, "Villa de Allende": 20.89, "Toluca": 4.39, "Metepec": 4.51},
    2016: {"Ensenada": 38.76, "Cajeme": 6.64, "Etchojoa": 6.69, "Zapopan": 6.13, "Villa de Allende": 21.45, "Toluca": 4.59, "Metepec": 4.42},
    2017: {"Ensenada": 49.04, "Cajeme": 6.38, "Etchojoa": 6.38, "Zapopan": 4.86, "Villa de Allende": 19.77, "Toluca": 4.66, "Metepec": 4.61},
    2018: {"Ensenada": 60.65, "Cajeme": 6.56, "Etchojoa": 6.69, "Zapopan": 7.67, "Villa de Allende": 20.74, "Toluca": 3.58, "Metepec": 3.40},
    2019: {"Ensenada": 74.16, "Cajeme": 7.08, "Etchojoa": 6.98, "Zapopan": 6.00, "Villa de Allende": 20.74, "Toluca": 4.82, "Metepec": 4.28},
    2020: {"Ensenada": 35.14, "Cajeme": 7.06, "Etchojoa": 6.75, "Zapopan": 6.96, "Villa de Allende": 22.00, "Toluca": 4.63, "Metepec": 4.30},
    2021: {"Ensenada": 43.87, "Cajeme": 7.41, "Etchojoa": 7.39, "Zapopan": 6.94, "Villa de Allende": 22.04, "Toluca": 4.48, "Metepec": 4.46},
    2022: {"Ensenada": 45.68, "Cajeme": 7.62, "Etchojoa": 6.87, "Zapopan": 7.40, "Villa de Allende": 21.36, "Toluca": 4.74, "Metepec": 4.10},
    2023: {"Ensenada": 38.41, "Cajeme": 7.90, "Etchojoa": 7.34, "Zapopan": 7.03, "Villa de Allende": 21.32, "Toluca": 4.62, "Metepec": 3.56},
    2024: {"Ensenada": 46.45, "Cajeme": 6.53, "Etchojoa": 6.29, "Zapopan": 8.17, "Villa de Allende": 21.88, "Toluca": 4.02, "Metepec": 3.78},
}

In [6]:
# ===========================================
# FUNCIONES FAO-56 + CONSTANTES + UTILIDADES
# ===========================================

# === Constantes FAO-56 (físicas y de referencia) ===
ALBEDO = 0.23         # Coef. de reflexión de la superficie de referencia (césped)
SIGMA  = 4.903e-9     # Constante de Stefan-Boltzmann [MJ K^-4 m^-2 día^-1]
P0_kPa = 101.3        # Presión atmosférica estándar a nivel del mar [kPa]
T0_K   = 293.0        # Temperatura estándar de referencia (20 °C en Kelvin)
GSC    = 0.0820       # Constante solar [MJ m^-2 min^-1]

# --- Saturación de vapor y pendientes ---
def es_kPa(T_C: float) -> float:
    """Presión de vapor de saturación (kPa). FAO-56 Eq. 11"""
    return 0.6108 * math.exp((17.27 * T_C) / (T_C + 237.3))

def delta_kPa_per_C(T_C: float) -> float:
    """Pendiente de la curva de saturación (kPa/°C). FAO-56 Eq. 13"""
    e = es_kPa(T_C)
    return 4098.0 * e / ((T_C + 237.3) ** 2)

# --- Presión y constante psicrométrica ---
def presion_atm_kPa(z_m: float) -> float:
    """
    Presión atmosférica (kPa) en función de la altitud (FAO-56).
    """
    #return P0_kPa * ((1.0 - 2.25577e-5 * z_m) ** 5.25588)
    return 101.3 * ((293.0 - 0.0065 * z_m) / 293.0) ** 5.26       # FAO 56 Eq.7    

def psicrometrica_gamma_kPa_per_C(P_kPa: float) -> float:
    """Constante psicrométrica (kPa/°C). FAO-56 Eq. 8"""
    return 0.000665 * P_kPa

# --- Radiación extraterrestre, cielo despejado y neta ---
def extraterrestrial_radiation_MJm2d(lat_deg: float, doy: int) -> float:
    """Ra [MJ m-2 d-1] — FAO-56 Eqs. 21–25."""
    phi = math.radians(lat_deg)
    dr  = 1.0 + 0.033 * math.cos(2.0 * math.pi / 365.0 * doy)
    delta = 0.409 * math.sin(2.0 * math.pi / 365.0 * doy - 1.39)
    ws = math.acos(-math.tan(phi) * math.tan(delta))
    Ra = (24.0 * 60.0 / math.pi) * GSC * dr * (
        ws * math.sin(phi) * math.sin(delta) +
        math.cos(phi) * math.cos(delta) * math.sin(ws)
    )
    return Ra

def clear_sky_radiation_Rso_FAO56(lat_deg: float, doy: int, z_m: float) -> float:
    """Rso [MJ m-2 d-1] — FAO-56 cielo despejado: (0.75 + 2e-5*z) * Ra."""
    Ra = extraterrestrial_radiation_MJm2d(lat_deg, doy)
    return (0.75 + 2e-5 * z_m) * Ra

def net_shortwave_Rns(Rs_MJm2d: float, albedo: float = ALBEDO) -> float:
    """Rns [MJ m-2 d-1] — (1 - albedo) * Rs (FAO-56)."""
    return (1.0 - albedo) * Rs_MJm2d

#============================ Versión FAO 56 ====================================
# def net_longwave_Rnl_FAO56(Tmax_C: float, Tmin_C: float, ea_kPa: float,
#                            Rs_MJm2d: float, Rso_MJm2d: float) -> float:
#     """
#     Rnl [MJ m-2 d-1] — FAO-56 Eq. 39.
#     Nota: NO usar la columna NASA 'ALLSKY_SFC_LW_DWN' (downward/entrante).
#     Se calcula con nubosidad: [1.35*(Rs/Rso) - 0.35] y se recorta 0 <= Rs/Rso <= 1.
#     """
#     tmaxK, tminK = Tmax_C + 273.16, Tmin_C + 273.16
#     t4 = (tmaxK**4 + tminK**4) / 2.0
#     rsrso = 0.0 if Rso_MJm2d <= 0.0 else max(0.0, min(Rs_MJm2d / Rso_MJm2d, 1.0))
#     fcloud = 1.35 * rsrso - 0.35
#     return SIGMA * t4 * (0.34 - 0.14 * math.sqrt(max(ea_kPa, 0.0))) * fcloud
#================================================================================
    

# =================== Versión "UAMEX" (comentar la FAO de arriba y descomentar ESTA) ===================
def net_longwave_Rnl_FAO56(Tmax_C: float, Tmin_C: float, ea_kPa: float,
                           Rs_MJm2d: float, Ra_o_Rso: float) -> float:
    """
    Rnl [MJ m-2 d-1] — Versión del otro equipo.
    DIFERENCIAS vs FAO:
      - Usa Ra (radiación extraterrestre) en lugar de Rso (cielo despejado).
      - NO recorta (clip) el cociente Rs/Ra al rango [0, 1].
    Importante: el 5º parámetro aquí debe ser **Ra**, no Rso.
    """
    tmaxK, tminK = Tmax_C + 273.16, Tmin_C + 273.16
    t4 = (tmaxK**4 + tminK**4) / 2.0
    nub = 1.35 * (Rs_MJm2d / Ra_o_Rso) - 0.35   # SIN clip 0..1
    return SIGMA * t4 * (0.34 - 0.14 * math.sqrt(max(ea_kPa, 0.0))) * nub
#================================================================================

# --- Penman-Monteith con G=0 a escala diaria ---
def eto_fao56_mm(Tmax, Tmin, RH_pct, u2_ms, Rs_MJm2d, lat_deg, z_m, doy, G_MJm2d: float = 0.0):
    """
    ETo [mm/día] — FAO-56 Penman-Monteith (Eq. 6) diario (G=0 por defecto).
    - Usa es, ea (de T y HR)
    - delta(Tmean), gamma(z)
    - Rn = Rns - Rnl (Rnl FAO-56 Eq. 39)
    """
    Tmean = (Tmax + Tmin) / 2.0
    es = (es_kPa(Tmax) + es_kPa(Tmin)) / 2.0
    ea = es * (RH_pct / 100.0)
    delta = delta_kPa_per_C(Tmean)
    P = presion_atm_kPa(z_m)
    gamma = psicrometrica_gamma_kPa_per_C(P)
    
    #===================== Versión FAO-56 ========================
    #Rso = clear_sky_radiation_Rso_FAO56(lat_deg, doy, z_m)
    #=============================================================

    #===================== Versión "UAMEX" =======================
    Rso = extraterrestrial_radiation_MJm2d(lat_deg, doy)   # ← aquí va Ra cuando uses la versión "equipo"

    Rns = net_shortwave_Rns(Rs_MJm2d=Rs_MJm2d, albedo=ALBEDO)
    Rnl = net_longwave_Rnl_FAO56(Tmax, Tmin, ea, Rs_MJm2d, Rso)
    Rn  = Rns - Rnl
    num = 0.408 * delta * (Rn - G_MJm2d) + gamma * (900.0 / (Tmean + 273.0)) * u2_ms * (es - ea)
    den = delta + gamma * (1.0 + 0.34 * u2_ms)
    ETo = max(0.0, num / den)
    return {
        "Tmean_": Tmean, "es_": es, "ea_": ea, "delta_": delta,
        "P_": P, "gamma_": gamma, "Rso_": Rso, "Rns_": Rns, "Rnl_": Rnl, "Rn_": Rn,
        "ET0": ETo
    }

#====================== Kc diario ===========================
# ===========================================================
# OPCIÓN 1 (ESCALONADA) — DESCOMENTAR PARA USAR RAMPAS LINEALES
# ===========================================================
# def curva_kc_diaria(
#     fechas_idx: pd.DatetimeIndex,
#     kc_ini: float, kc_mid: float, kc_end: float,
#     dur,  # (d_ini, d_des, d_mid, d_fin)
#     endpoint_rampas: bool = True
# ) -> pd.Series:
#     """
#     Versión ESCALONADA (con rampas lineales):
#       - Inicio: kc_ini (constante)
#       - Desarrollo: rampa lineal kc_ini → kc_mid
#       - Media: kc_mid (constante)
#       - Final: rampa lineal kc_mid → kc_end
#     Parámetro:
#       endpoint_rampas=True incluye el valor final de la rampa (puede duplicar en uniones).
#     """
#     d_ini, d_des, d_mid, d_fin = dur
#     n = len(fechas_idx)
#     kc_ini_arr  = np.full(d_ini, kc_ini)
#     kc_des_arr  = np.linspace(kc_ini, kc_mid, d_des, endpoint=endpoint_rampas)
#     kc_mid_arr  = np.full(d_mid, kc_mid)
#     kc_fin_arr  = np.linspace(kc_mid, kc_end, d_fin, endpoint=endpoint_rampas)
#     kc_full = np.concatenate([kc_ini_arr, kc_des_arr, kc_mid_arr, kc_fin_arr])
#     if len(kc_full) >= n:
#         kc_full = kc_full[:n]
#     else:
#         kc_full = np.concatenate([kc_full, np.full(n - len(kc_full), kc_end)])
#     return pd.Series(kc_full, index=fechas_idx, name="Kc_")

# ===========================================================
# OPCIÓN 2 (ASIGNACIÓN POR FASE, CON SALTOS) — ACTIVA POR DEFECTO
# ===========================================================
def curva_kc_diaria(
    fechas_idx: pd.DatetimeIndex,
    kc_ini: float, kc_mid: float, kc_end: float,
    dur,  # (d_ini, d_des, d_mid, d_fin)
    endpoint_rampas: bool = True  # se mantiene por compatibilidad, no se usa
) -> pd.Series:
    """
    Versión con ASIGNACIÓN POR FASE (sin rampas):
      - Inicio: kc_ini (constante)
      - Desarrollo: kc_mid (constante)  ← se permiten saltos
      - Media: kc_mid (constante)
      - Final: kc_end (constante)
    """
    d_ini, d_des, d_mid, d_fin = dur
    n = len(fechas_idx)

    kc_ini_arr = np.full(d_ini, kc_ini)
    kc_des_arr = np.full(d_des, kc_mid)   # salto kc_ini -> kc_mid
    kc_mid_arr = np.full(d_mid, kc_mid)
    kc_fin_arr = np.full(d_fin, kc_end)   # salto kc_mid -> kc_end

    kc_full = np.concatenate([kc_ini_arr, kc_des_arr, kc_mid_arr, kc_fin_arr])

    # Ajuste a la longitud disponible
    if len(kc_full) >= n:
        kc_full = kc_full[:n]
    else:
        kc_full = np.concatenate([kc_full, np.full(n - len(kc_full), kc_end)])

    return pd.Series(kc_full, index=fechas_idx, name="Kc_")
#================================================================
    
# --- Pef FAO simple ---
def pef_simple_FAO_por_dia(Ptot_series_mm: pd.Series) -> pd.Series:
    """
    FAO simple: Pef = f * Ptotal.
    f = 0.8 si Ptotal < 250 mm; f = 0.6 si no.
    Se aplica el mismo factor f a cada día.
    """
    Ptotal = Ptot_series_mm.fillna(0).sum()
    f = 0.8 if Ptotal < 250.0 else 0.6
    return Ptot_series_mm.fillna(0) * f  # Pef_ diario    

def rendimiento_por_archivo(df: pd.DataFrame, region: str) -> float:
    """
    Obtiene el rendimiento (ton/ha) para el/los años presentes en el archivo.
    - Si hay varios años, usa el promedio simple de los disponibles en RENDIMIENTOS_TON.
    - Si no hay coincidencias, devuelve NaN.
    """
    years = sorted(set(int(y) for y in pd.Series(df["YEAR"]).dropna().unique()))
    rend_vals = []
    for y in years:
        try:
            rend = RENDIMIENTOS_TON[y][region]
            rend_vals.append(float(rend))
        except KeyError:
            # año o región no disponibles en el diccionario
            pass
    if rend_vals:
        return float(np.mean(rend_vals))
    return float("nan")

def calcular_uac_hh(
    ETverde_series_mm: pd.Series,
    ETazul_series_mm: pd.Series,
    rendimiento_ton_ha: float
) -> dict:
    """
    Calcula:
      - ETverde_total_mm, ETazul_total_mm
      - UACverde_m3_ha, UACazul_m3_ha
      - HHverde_m3_ton, HHazul_m3_ton
    Convenciones:
      - 1 mm sobre 1 ha = 10 m3  => UAC = sum(ET) * 10
      - HH = UAC / rendimiento(ton/ha)
    Si no hay rendimiento válido (>0), HH = NaN.
    """
    ETverde_total_mm = float(pd.Series(ETverde_series_mm).fillna(0).sum())
    ETazul_total_mm  = float(pd.Series(ETazul_series_mm).fillna(0).sum())

    UACverde_m3_ha = ETverde_total_mm * 10.0
    UACazul_m3_ha  = ETazul_total_mm  * 10.0

    if (rendimiento_ton_ha is not None) and (rendimiento_ton_ha > 0):
        HHverde_m3_ton = UACverde_m3_ha / rendimiento_ton_ha
        HHazul_m3_ton  = UACazul_m3_ha  / rendimiento_ton_ha
    else:
        HHverde_m3_ton = float("nan")
        HHazul_m3_ton  = float("nan")

    return {
        "ETverde_total_mm": ETverde_total_mm,
        "ETazul_total_mm":  ETazul_total_mm,
        "UACverde_m3_ha":   UACverde_m3_ha,
        "UACazul_m3_ha":    UACazul_m3_ha,
        "HHverde_m3_ton":   HHverde_m3_ton,
        "HHazul_m3_ton":    HHazul_m3_ton,
    }

# --- Década del ciclo (1..ceil(n/10)) ---
def decada_del_ciclo(idx: pd.DatetimeIndex) -> pd.Series:
    """Décadas relativas al ciclo (no calendario)."""
    n = len(idx)
    dec = np.ceil((np.arange(1, n+1)) / 10.0).astype(int)
    return pd.Series(dec, index=idx, name="decada_")

# --- construir fechas a partir de YEAR + DOY ---
def fecha_desde_year_doy(year: int, doy: int) -> datetime:
    """Fecha calendario a partir de año (YEAR) y día juliano (DOY)."""
    return datetime(year, 1, 1) + timedelta(days=int(doy) - 1)


# Radiación Neta (Rn) según FAO-56

## 1) Definición de Rn

En FAO-56, la radiación neta diaria se calcula como:

\[
R_n = R_{ns} - R_{nl}
\]

- **\(R_{ns}\)**: radiación neta de onda corta  
\[
R_{ns} = (1 - \alpha)R_s
\]  
con \(\alpha\) (albedo) ≈ 0.23 para referencia (gramínea corta).

- **\(R_{nl}\)**: radiación neta de onda larga (Ecuación 39 FAO-56)  
\[
R_{nl} = \sigma \frac{(T_{max,K}^4 + T_{min,K}^4)}{2} (0.34 - 0.14 e_a)\,[1.35(R_s / R_{so}) - 0.35]
\]

donde:  
- \(\sigma = 4.903 \times 10^{-9}\) MJ K\(^{-4}\) m\(^{-2}\) día\(^{-1}\)  
- \(e_a\) = presión real de vapor (kPa)  
- \(R_s\) = radiación solar global medida/modelada (NASA `ALLSKY_SFC_SW_DWN`)  
- \(R_{so}\) = radiación de cielo despejado (clear-sky)

**Clave:** en FAO-56 no se usa la radiación de onda larga *entrante* (`ALLSKY_SFC_LW_DWN`).  
El \(R_{nl}\) se estima con la ecuación anterior usando \(R_s/R_{so}\) como indicador de nubosidad.

---

- `ALLSKY_SFC_LW_DWN` = onda larga entrante.  
- FAO-56 requiere \(R_{nl}\) neto saliente (saliente – entrante vía fórmula).  
- Si se mezcla mal esta señal, \(R_{nl}\) queda mal escalado ⇒ \(R_n\) distinto.  

NO usar `ALLSKY_SFC_LW_DWN` en el cálculo. Solo se guarda como trazabilidad.

---

In [7]:
# ==== LECTOR NASA (POWER) + MAPEOS ====

# Columnas que esperamos en la tabla NASA
COLS_MIN = ["YEAR","DOY","ALLSKY_SFC_SW_DWN","ALLSKY_SFC_LW_DWN","PRECTOTCORR",
            "WS2M","T2M_MAX","T2M_MIN","RH2M"]

def leer_nasa_power(path_csv: Path) -> pd.DataFrame:
    """
    Lee CSV de NASA/POWER con -BEGIN/-END HEADER-.
    - Tolera BOM (utf-8 / utf-8-sig).
    - Detecta la fila de cabeceras (línea que contiene YEAR y DOY).
    - Usa sep=None con engine='python' para que pandas infiera el delimitador 
      (funciona con coma, tab, punto y coma o espacios múltiples).
    - Normaliza nombres y valida columnas mínimas.
    """
    import re
    from io import StringIO

    # 1) lee texto crudo tolerando BOM
    try:
        raw = path_csv.read_text(encoding="utf-8")
    except UnicodeDecodeError:
        raw = path_csv.read_text(encoding="utf-8-sig")

    # 2) encuentra la línea de cabeceras
    header_idx = None
    lines = raw.splitlines()
    for i, ln in enumerate(lines):
        st = ln.lstrip("\ufeff").strip()
        if st.startswith("YEAR"):
            header_idx = i
            break
    if header_idx is None:
        for i, ln in enumerate(lines):
            toks = re.split(r"[,\t; ]+", ln.strip())
            if {"YEAR", "DOY"}.issubset(set(toks)):
                header_idx = i
                break
    if header_idx is None:
        raise ValueError(f"No se encontró encabezado de columnas en: {path_csv.name}")

    data_str = "\n".join(lines[header_idx:])

    # 3) lectura con inferencia automática de delimitador
    df = pd.read_csv(StringIO(data_str), sep=None, engine="python")

    # 4) normaliza nombres y corrige variantes típicas
    def _norm_col(s: str) -> str:
        s = str(s).replace("\ufeff", "").strip()
        s = re.sub(r"[ \t]+", " ", s)
        return s

    df.columns = [_norm_col(c) for c in df.columns]
    fix = {
        "ALLSKY SFC SW DWN": "ALLSKY_SFC_SW_DWN",
        "ALLSKY SFC LW DWN": "ALLSKY_SFC_LW_DWN",
    }
    df.rename(columns={k: v for k, v in fix.items() if k in df.columns}, inplace=True)

    # 5) validación mínima
    faltan = [c for c in COLS_MIN if c not in df.columns]
    if faltan:
        ejemplos = ", ".join(list(df.columns)[:12])
        raise ValueError(f"{path_csv.name}: faltan columnas {faltan}. "
                         f"Detectadas: {ejemplos}")

    return df

# Mapeo de nombres (salida): "NombreDeEllos (NombreNASA)" o con sufijo "_"
MAP_NOMBRES = {
    "YEAR":              "Año_ (YEAR)",
    "DOY":               "Día (DOY)",
    "T2M_MAX":           "Tmax (T2M_MAX)",
    "T2M_MIN":           "Tmin (T2M_MIN)",
    "RH2M":              "HR (RH2M)",
    "WS2M":              "Ux (WS2M)",
    "ALLSKY_SFC_SW_DWN": "Rs (ALLSKY_SFC_SW_DWN)",
    "ALLSKY_SFC_LW_DWN": "Rl_ (ALLSKY_SFC_LW_DWN)",  # sólo trazabilidad; NO entra a Rnl FAO
    "PRECTOTCORR":       "Ptot_ (PRECTOTCORR)",
}



In [9]:
# ==========================================================
#  PROCESAR TODO (por región y archivo) + EXPORTAR
# ==========================================================

def procesar_archivo(region: str, ruta_archivo: Path) -> Path | None:
    info = REGIONES[region]
    lat, z_m = info["lat"], info["z_m"]
    kc_ini, kc_mid, kc_end = info["kc"]["ini"], info["kc"]["mid"], info["kc"]["end"]
    dur = info["dur"]

    try:
        df = leer_nasa_power(ruta_archivo)
    except Exception as e:
        print(f"[{region}] {ruta_archivo.name}: {e}. Salto.")
        return None

    # --- fechas y orden ---
    df["date"] = [fecha_desde_year_doy(int(y), int(d)) for y, d in zip(df["YEAR"], df["DOY"])]
    df = df.sort_values("date").reset_index(drop=True)
    fechas = pd.DatetimeIndex(df["date"])

    # --- Kc diario (índice consistente) ---
    kc_series = curva_kc_diaria(fechas, kc_ini, kc_mid, kc_end, dur)
    kc_series = kc_series.reset_index(drop=True)  # <-- igual que el df

    # --- ETo FAO-56 por fila ---
    eto_out = []
    for _, r in df.iterrows():
        calc = eto_fao56_mm(
            Tmax=float(r["T2M_MAX"]),
            Tmin=float(r["T2M_MIN"]),
            RH_pct=float(r["RH2M"]),
            u2_ms=float(r["WS2M"]),
            Rs_MJm2d=float(r["ALLSKY_SFC_SW_DWN"]),
            lat_deg=float(lat),
            z_m=float(z_m),
            doy=int(r["DOY"]),
        )
        eto_out.append(calc)
    eto_df = pd.DataFrame(eto_out, index=df.index)  # RangeIndex 0..n-1

    # --- Pef (FAO simple sobre el ciclo) ---
    Ptot_series = df["PRECTOTCORR"].astype(float)
    pef_series  = pef_simple_FAO_por_dia(Ptot_series)
    # Asegurar índice consistente con kc_series/df:
    pef_series.index = df.index  # <-- clave

    # --- Década del ciclo (1..ceil(n/10)) con índice consistente ---
    decada_series = decada_del_ciclo(fechas).reset_index(drop=True)  # <-- clave

    # --- ETc, ET verde/azul (todo con el mismo índice) ---
    ET0 = eto_df["ET0"]                      # RangeIndex
    Kc_ = kc_series                          # RangeIndex
    ETc = ET0 * Kc_                          # sin desalineo
    ETverde = np.minimum(ETc, pef_series)    # índices iguales -> sin warnings
    ETazul  = np.maximum(ETc - pef_series, 0.0)

    # --- Rendimiento (ton/ha) según año(s) presentes en el archivo ---
    rendimiento_ton_ha = rendimiento_por_archivo(df, region)

    # --- Resumen UAC/HH (m3/ha y m3/ton) para el archivo ---
    resumen_hh = calcular_uac_hh(
        ETverde_series_mm=ETverde,
        ETazul_series_mm=ETazul,
        rendimiento_ton_ha=rendimiento_ton_ha
    )

    # Log de resumen en consola
#    print(f"[{region}] {ruta_archivo.name} -> Rend(t/ha)={rendimiento_ton_ha:.3f} | "
#          f"ETv={resumen_hh['ETverde_total_mm']:.2f} mm, ETa={resumen_hh['ETazul_total_mm']:.2f} mm | "
#          f"UACv={resumen_hh['UACverde_m3_ha']:.1f} m3/ha, UACa={resumen_hh['UACazul_m3_ha']:.1f} m3/ha | "
#          f"HHv={resumen_hh['HHverde_m3_ton']:.1f} m3/ton, HHa={resumen_hh['HHazul_m3_ton']:.1f} m3/ton")

    # --- Construir salida con nombres solicitados ---
    base_cols = ["YEAR","DOY","T2M_MAX","T2M_MIN","RH2M","WS2M",
                 "ALLSKY_SFC_SW_DWN","ALLSKY_SFC_LW_DWN","PRECTOTCORR"]
    out = df[base_cols].copy()
    out.columns = [MAP_NOMBRES[c] for c in base_cols]

    # Agregados (generados con sufijo "_", sin paréntesis)
    out["Pef_"]   = pef_series.values
    for col in ["Tmean_","es_","ea_","delta_","P_","gamma_","Rns_","Rnl_","Rn_"]:
        out[col] = eto_df[col].values
    out["Rso_"] = eto_df["Rso_"].values  # trazabilidad

    # Kc, decada, ET0/ETc/verdes/azules
    out["Kc_"]     = Kc_.values
    out["decada_"] = decada_series.values
    out["ET0"]     = ET0.values
    out["ETc"]     = ETc.values
    out["ETverde"] = ETverde.values
    out["ETazul"]  = ETazul.values

    # ===== Resumen UAC/HH (constante por fila, útil para adjuntar al CSV) =====
    out["UACverde_m3_ha"] = resumen_hh["UACverde_m3_ha"]
    out["UACazul_m3_ha"]  = resumen_hh["UACazul_m3_ha"]
    out["HHverde_m3_ton"] = resumen_hh["HHverde_m3_ton"]
    out["HHazul_m3_ton"]  = resumen_hh["HHazul_m3_ton"]

    # --- guardar ---
    out_dir = RUTA_SALIDA / region
    out_dir.mkdir(parents=True, exist_ok=True)
    out_name = ruta_archivo.stem + "-SALIDA.csv"
    out_path = out_dir / out_name
    out.to_csv(out_path, index=False)
    return out_path


# ======== Ejecutar en todas las regiones ========
exportados = []
for region_dir in sorted(RUTA_ENTRADA.iterdir()):
    if not region_dir.is_dir():
        continue
    region = region_dir.name
    if region not in REGIONES:
        print(f"⚠️ Región no conocida en REGIONES: {region}. Salto.")
        continue

    count_ok = 0
    for f in sorted(region_dir.glob("*.csv")):
        outp = procesar_archivo(region, f)
        if outp is not None:
            count_ok += 1
            exportados.append(outp)
    print(f"✅ {region}: {count_ok} archivos procesados.")

✅ Cajeme: 14 archivos procesados.
✅ Ensenada: 14 archivos procesados.
✅ Etchojoa: 14 archivos procesados.
✅ Metepec: 15 archivos procesados.
✅ Toluca: 15 archivos procesados.
✅ Villa de Allende: 15 archivos procesados.
✅ Zapopan: 15 archivos procesados.
